In [1]:

import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm
#import datetime
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import csv
import time

from yahooquery import Ticker


#import pandas_datareader.data as data
#import yfinance as yf


#yf.pdr_override()
from scipy import stats


import mibian

import sys
sys.path.append(r'D:\code')
#C:\Users\jgavarrete\daily

np.seterr(divide='ignore', invalid='ignore')

import requests
import json


In [2]:
ape_url = 'https://apewisdom.io/api/v1.0/filter/wallstreetbets'

ape_stonks = pd.DataFrame([])

new_results = True
#df =[]
df1 = []

page = 1

while new_results:

    res = requests.get(ape_url + f"/page/{page}").json()

    new_results = res.get("results", [])
    df = pd.DataFrame.from_dict(new_results)
    #df1 = pd.DataFrame.from_records(df.results)

    ape_stonks = ape_stonks.append(df)
    page += 1

In [3]:
ape_stonks['mentions_change'] = (ape_stonks.mentions.astype(float)/ape_stonks.mentions_24h_ago.astype(float) -1)

In [4]:
#conditions on mentions

cond1 = ape_stonks.mentions_change >= .8
cond2= ape_stonks.mentions_change <= -.70




In [5]:
#ape_stonks2 = ape_stonks[cond1 | cond2]
ape_stonks2 = ape_stonks[cond1]

In [6]:
ape_stonks

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago,mentions_change
0,1,CHWY,Chewy,499,3026,7,105,3.752381
1,2,GME,GameStop,485,2930,1,780,-0.378205
2,3,SPY,SPDR S&amp;P 500 ETF Trust,465,1810,2,494,-0.058704
3,4,TSLA,Tesla,422,2781,4,331,0.274924
4,5,AAPL,Apple,241,927,3,357,-0.324930
...,...,...,...,...,...,...,...,...
95,996,CVE,Cenovus Energy,1,2,632,1,0.000000
96,997,BAM,Brookfield Asset Management,1,1,636,None,NaN
97,998,MG,MISTRAS Group,1,1,600,None,NaN
98,999,WB,Weibo,1,4,418,1,0.000000


In [7]:
#amount of tickers
ape_list = ape_stonks.ticker.to_list()[:10]

In [8]:
#load stock list from spreadsheet

stock_list1 = pd.read_csv(r".\stock_list.csv",header=None)
stonks=stock_list1.iloc[:,0].tolist()


In [10]:
#make vairable stonks = to ape_list if want to use list from query

stonks=ape_list

In [11]:
len(stonks)

10

In [12]:
(len(stonks)/4)

2.5

In [13]:
#if list of tickers is too long, split into 4 so yahoo pulls dont get rejected
# n = (len(stonks)/4) rounded up to nearest integer
n=3

list1 = list(stonks[0:n])
list2 = list(stonks[n:2*n])
list3 = list(stonks[2*n:3*n])
list4 = list(stonks[3*n:4*n])
#list5 = list(stonks[4*n:5*n])

In [14]:
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))
#print(len(list5))

3
3
3
1


In [16]:
stocklist = stonks


In [17]:
t1 = Ticker(list1, asynchronous=True)
t2 = Ticker(list2, asynchronous=True)
t3 = Ticker(list3, asynchronous=True)
t4 = Ticker(list4, asynchronous=True)
#t5 = Ticker(list5, asynchronous=True)

#twt = Ticker(stonks1, asynchronous=True)


In [18]:
# delay = time between data pulls, the longer the list, the more time you will need to allow in between pulls
# code below pulls all option chains for all selected tickers
delay = 1

start = time.time()

df_1 = pd.DataFrame()
df_1 = t1.option_chain

time.sleep(delay)   

df_2 = pd.DataFrame()
df_2 = t2.option_chain

time.sleep(delay) 

df_3 = pd.DataFrame()
df_3 = t3.option_chain

time.sleep(delay) 

df_4 = pd.DataFrame()
df_4 = t4.option_chain

end = time.time()
print(end - start)

8.446978330612183


In [19]:
# 
df = pd.concat([df_1,df_2,df_3,df_4])#df_5])  #ignore_index=True)

In [20]:
df.shape

(27952, 14)

In [21]:
df

contractSymbol  strike currency  lastPrice  \
symbol expiration optionType                                                    
CHWY   2021-12-10 calls       CHWY211210C00030000    30.0      USD      27.06   
                  calls       CHWY211210C00035000    35.0      USD      22.51   
                  calls       CHWY211210C00040000    40.0      USD      16.83   
                  calls       CHWY211210C00045000    45.0      USD      12.69   
                  calls       CHWY211210C00050000    50.0      USD       8.35   
...                                           ...     ...      ...        ...   
LCID   2024-01-19 puts        LCID240119P00065000    65.0      USD      37.35   
                  puts        LCID240119P00070000    70.0      USD      39.30   
                  puts        LCID240119P00075000    75.0      USD      46.47   
                  puts        LCID240119P00080000    80.0      USD      50.42   
                  puts        LCID240119P00085000    85.0      USD      54.98   

                                 change  volume    bid    ask contractSize  \
symbol expiration optionType                                                 
CHWY   2021-12-10 calls       27.060000    35.0  25.95  27.65      REGULAR   
                  calls       22.510000     1.0  21.25  23.65      REGULAR   
                  calls       -3.469999    34.0  16.20  17.55      REGULAR   
                  calls       -3.310000   112.0  11.90  13.00      REGULAR   
                  calls       -3.000000   195.0   8.20   8.55      REGULAR   
...                                 ...     ...    ...    ...          ...   
LCID   2024-01-19 puts        -2.650002     1.0  36.50  41.40      REGULAR   
                  puts         0.000000     2.0  41.00  45.50      REGULAR   
                  puts         0.540001     1.0  45.00  49.90      REGULAR   
                  puts         0.000000     2.0  49.50  54.40      REGULAR   
                  puts         2.410000     1.0  54.00  58.90      REGULAR   

                                   lastTradeDate  impliedVolatility  \
symbol expiration optionType                                          
CHWY   2021-12-10 calls      2021-12-09 20:56:50           8.023442   
                  calls      2021-12-09 20:56:50           7.953125   
                  calls      2021-12-09 20:59:32           5.046879   
                  calls      2021-12-09 20:56:00           4.605473   
                  calls      2021-12-09 20:59:35           4.109380   
...                                          ...                ...   
LCID   2024-01-19 puts       2021-12-09 15:08:44           0.805788   
                  puts       2021-12-06 15:48:16           0.804323   
                  puts       2021-12-09 17:00:03           0.795168   
                  puts       2021-12-06 14:37:10           0.797487   
                  puts       2021-12-09 17:00:03           0.797487   

                              inTheMoney  percentChange  openInterest  
symbol expiration optionType                                           
CHWY   2021-12-10 calls             True       0.000000           0.0  
                  calls             True       0.000000           0.0  
                  calls             True     -17.093594           2.0  
                  calls             True     -20.687502          13.0  
                  calls             True     -26.431719         175.0  
...                                  ...            ...           ...  
LCID   2024-01-19 puts              True      -6.625004          11.0  
                  puts              True       0.000000          15.0  
                  puts              True       1.175704          26.0  
                  puts              True       0.000000         884.0  
                  puts              True       4.584364         915.0  

[27952 rows x 14 columns]

In [22]:
df.reset_index(inplace=True)

In [23]:
# to check for a specific ticker
df.loc[df['symbol'] == 'LCID']

,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,volume,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney,percentChange,openInterest
26969,LCID,2021-12-10,calls,LCID211210C00020000,20.0,USD,16.50,-8.500000,1796.0,16.45,16.75,REGULAR,2021-12-09 20:58:46,5.906253,True,-34.000000,7.0
26970,LCID,2021-12-10,calls,LCID211210C00022000,22.0,USD,21.70,0.000000,1.0,14.15,15.30,REGULAR,2021-12-07 17:00:02,6.046877,True,0.000000,30.0
26971,LCID,2021-12-10,calls,LCID211210C00023000,23.0,USD,14.60,-6.900000,1.0,13.40,14.30,REGULAR,2021-12-09 19:53:05,6.234377,True,-32.093020,23.0
26972,LCID,2021-12-10,calls,LCID211210C00024000,24.0,USD,19.15,0.000000,6.0,12.25,13.15,REGULAR,2021-12-08 15:37:33,5.015629,True,0.000000,17.0
26973,LCID,2021-12-10,calls,LCID211210C00025000,25.0,USD,12.15,-8.050001,35.0,11.40,11.70,REGULAR,2021-12-09 20:26:21,3.375002,True,-39.851490,781.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27947,LCID,2024-01-19,puts,LCID240119P00065000,65.0,USD,37.35,-2.650002,1.0,36.50,41.40,REGULAR,2021-12-09 15:08:44,0.805788,True,-6.625004,11.0
27948,LCID,2024-01-19,puts,LCID240119P00070000,70.0,USD,39.30,0.000000,2.0,41.00,45.50,REGULAR,2021-12-06 15:48:16,0.804323,True,0.000000,15.0
27949,LCID,2024-01-19,puts,LCID240119P00075000,75.0,USD,46.47,0.540001,1.0,45.00,49.90,REGULAR,2021-12-09 17:00:03,0.795168,True,1.175704,26.0
27950,LCID,2024-01-19,puts,LCID240119P00080000,80.0,USD,50.42,0.000000,2.0,49.50,54.40,REGULAR,2021-12-06 14:37:10,0.797487,True,0.000000,884.0


In [24]:
stocks=df.symbol.unique().tolist() 
stocks2 = Ticker(stocks, asynchronous=True)

In [25]:
# function to get latest stock prices

def getquotetoday(symbol):
        tk= Ticker(symbol).price
        quote=tk[symbol]['regularMarketPrice']
        return quote




In [26]:
# get latest stock prices
start = time.time()
L2=list([getquotetoday(item) for item in stocks])
end = time.time()
print(end - start)




1.530686616897583


In [27]:
# create dict with ticker and latest stock price
d = dict(zip(stocks,L2))

In [30]:
# map stock prices to main dataframe
df['stock_price'] = df['symbol'].map(d)

In [31]:
df

,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,volume,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney,percentChange,openInterest,stock_price
0,CHWY,2021-12-10,calls,CHWY211210C00030000,30.0,USD,27.06,27.060000,35.0,25.95,27.65,REGULAR,2021-12-09 20:56:50,8.023442,True,0.000000,0.0,56.30
1,CHWY,2021-12-10,calls,CHWY211210C00035000,35.0,USD,22.51,22.510000,1.0,21.25,23.65,REGULAR,2021-12-09 20:56:50,7.953125,True,0.000000,0.0,56.30
2,CHWY,2021-12-10,calls,CHWY211210C00040000,40.0,USD,16.83,-3.469999,34.0,16.20,17.55,REGULAR,2021-12-09 20:59:32,5.046879,True,-17.093594,2.0,56.30
3,CHWY,2021-12-10,calls,CHWY211210C00045000,45.0,USD,12.69,-3.310000,112.0,11.90,13.00,REGULAR,2021-12-09 20:56:00,4.605473,True,-20.687502,13.0,56.30
4,CHWY,2021-12-10,calls,CHWY211210C00050000,50.0,USD,8.35,-3.000000,195.0,8.20,8.55,REGULAR,2021-12-09 20:59:35,4.109380,True,-26.431719,175.0,56.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27947,LCID,2024-01-19,puts,LCID240119P00065000,65.0,USD,37.35,-2.650002,1.0,36.50,41.40,REGULAR,2021-12-09 15:08:44,0.805788,True,-6.625004,11.0,36.52
27948,LCID,2024-01-19,puts,LCID240119P00070000,70.0,USD,39.30,0.000000,2.0,41.00,45.50,REGULAR,2021-12-06 15:48:16,0.804323,True,0.000000,15.0,36.52
27949,LCID,2024-01-19,puts,LCID240119P00075000,75.0,USD,46.47,0.540001,1.0,45.00,49.90,REGULAR,2021-12-09 17:00:03,0.795168,True,1.175704,26.0,36.52
27950,LCID,2024-01-19,puts,LCID240119P00080000,80.0,USD,50.42,0.000000,2.0,49.50,54.40,REGULAR,2021-12-06 14:37:10,0.797487,True,0.000000,884.0,36.52


In [32]:
# Black Scholes function

def optionBS_price(optionType,S, K, sigma, dte):
    '''optionType=call or put, S=stock_price,K=strike, sigma=newMeanSigma, dte=dte .'''
    r=0
    t=dte/365
    d1 = np.multiply( 1. / sigma * np.divide(1., np.sqrt(t)),
        np.log(S/K) + (r + sigma**2 / 2.) * t  )
    d2 = d1 - sigma * np.sqrt(t)
    
    if optionType=="calls":
        C = np.multiply(S, norm.cdf(d1)) - \
        np.multiply(norm.cdf(d2) * K, np.exp(-r * t))
        return C
       
    if optionType=="puts":
        P = -np.multiply(S, norm.cdf(-d1)) + \
        np.multiply(norm.cdf(-d2) * K, np.exp(-r * t))
        return P

In [33]:
# create some new fields

df['V/OI']= df['volume'].astype('float')/df['openInterest']
df['contractSymbol'] = df['contractSymbol'].astype(str)

df[['bid', 'ask', 'strike']] = df[['bid', 'ask', 'strike']].apply(pd.to_numeric)
df['mark'] = (df['bid'] + df['ask']) / 2 # Calculate the midpoint of the bid-ask

        
df.insert(3, 'Distance OTM', df['stock_price'] - df['strike'])
df['Value'] = df['openInterest']*df['lastPrice']*100
df['Dist/Price'] = -df['Distance OTM'] / df['stock_price'] 

df['dte'] = (df['expiration'] - datetime.today()).dt.days 
df = df.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])

In [34]:
df

,symbol,expiration,optionType,Distance OTM,contractSymbol,strike,lastPrice,volume,bid,ask,impliedVolatility,inTheMoney,openInterest,stock_price,V/OI,mark,Value,Dist/Price,dte
0,CHWY,2021-12-10,calls,26.30,CHWY211210C00030000,30.0,27.06,35.0,25.95,27.65,8.023442,True,0.0,56.30,inf,26.800,0.0,-0.467140,0
1,CHWY,2021-12-10,calls,21.30,CHWY211210C00035000,35.0,22.51,1.0,21.25,23.65,7.953125,True,0.0,56.30,inf,22.450,0.0,-0.378330,0
2,CHWY,2021-12-10,calls,16.30,CHWY211210C00040000,40.0,16.83,34.0,16.20,17.55,5.046879,True,2.0,56.30,17.000000,16.875,3366.0,-0.289520,0
3,CHWY,2021-12-10,calls,11.30,CHWY211210C00045000,45.0,12.69,112.0,11.90,13.00,4.605473,True,13.0,56.30,8.615385,12.450,16497.0,-0.200710,0
4,CHWY,2021-12-10,calls,6.30,CHWY211210C00050000,50.0,8.35,195.0,8.20,8.55,4.109380,True,175.0,56.30,1.114286,8.375,146125.0,-0.111901,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27947,LCID,2024-01-19,puts,-28.48,LCID240119P00065000,65.0,37.35,1.0,36.50,41.40,0.805788,True,11.0,36.52,0.090909,38.950,41085.0,0.779847,770
27948,LCID,2024-01-19,puts,-33.48,LCID240119P00070000,70.0,39.30,2.0,41.00,45.50,0.804323,True,15.0,36.52,0.133333,43.250,58950.0,0.916758,770
27949,LCID,2024-01-19,puts,-38.48,LCID240119P00075000,75.0,46.47,1.0,45.00,49.90,0.795168,True,26.0,36.52,0.038462,47.450,120822.0,1.053669,770
27950,LCID,2024-01-19,puts,-43.48,LCID240119P00080000,80.0,50.42,2.0,49.50,54.40,0.797487,True,884.0,36.52,0.002262,51.950,4457128.0,1.190581,770


In [35]:
df['Abs_Dist/Price'] = df['Dist/Price'].abs()

In [37]:
# get historical stock prices for n days

d = datetime.today() - timedelta(days=30)
yq_data = stocks2.history(start=d, interval='1d')

In [38]:
yq_data.reset_index(inplace=True)

In [39]:
yq_data

,symbol,date,close,open,high,low,volume,adjclose,dividends
0,CHWY,2021-11-10,72.150002,76.050003,76.055000,71.529999,3012900,72.150002,0.0
1,CHWY,2021-11-11,71.300003,73.250000,73.800003,71.169998,2624700,71.300003,0.0
2,CHWY,2021-11-12,76.910004,71.809998,77.080002,70.849998,3427400,76.910004,0.0
3,CHWY,2021-11-15,74.760002,77.029999,77.589996,74.401001,2107300,74.760002,0.0
4,CHWY,2021-11-16,75.980003,74.250000,76.580002,73.879997,2147200,75.980003,0.0
...,...,...,...,...,...,...,...,...,...
205,LCID,2021-12-03,47.270000,48.549999,49.490002,44.570000,76351400,47.270000,0.0
206,LCID,2021-12-06,44.860001,40.025002,45.750000,38.060001,103929800,44.860001,0.0
207,LCID,2021-12-07,43.880001,46.505001,47.250000,43.020000,57740800,43.880001,0.0
208,LCID,2021-12-08,44.720001,42.599998,45.400002,41.340000,50135900,44.720001,0.0


In [40]:
#get latest volume
today_volume = yq_data.groupby('symbol')['volume'].last().to_dict()

In [41]:
# drop columns that are not used
yq_data = yq_data.drop(columns = ['open', 'low', 'close', 'high'])# 'dividends'])

In [42]:
# calculate log return using daily historical price data
yq_data = yq_data.assign(logret=np.log(yq_data.adjclose).groupby(yq_data.symbol).diff()) 

In [43]:
yq_data

,symbol,date,volume,adjclose,dividends,logret
0,CHWY,2021-11-10,3012900,72.150002,0.0,NaN
1,CHWY,2021-11-11,2624700,71.300003,0.0,-0.011851
2,CHWY,2021-11-12,3427400,76.910004,0.0,0.075740
3,CHWY,2021-11-15,2107300,74.760002,0.0,-0.028353
4,CHWY,2021-11-16,2147200,75.980003,0.0,0.016187
...,...,...,...,...,...,...
205,LCID,2021-12-03,76351400,47.270000,0.0,-0.023831
206,LCID,2021-12-06,103929800,44.860001,0.0,-0.052329
207,LCID,2021-12-07,57740800,43.880001,0.0,-0.022088
208,LCID,2021-12-08,50135900,44.720001,0.0,0.018962


In [44]:
# new dataframe using for the standard deviation of log returns
yq_data2 =pd.DataFrame()
yq_data2 = yq_data.groupby('symbol').agg(np.std, ddof=1) 

In [45]:
yq_data2

,date,volume,adjclose,dividends,logret
symbol,,,,,
AAPL,NaN,3.448476e+07,8.060378,0.000000,0.016358
AMC,NaN,1.403760e+07,5.109917,0.000000,0.058931
AMD,NaN,1.098794e+07,6.494013,0.000000,0.033781
CHWY,NaN,1.709795e+06,5.789961,0.000000,0.039167
GME,NaN,1.204694e+06,21.699732,0.000000,0.058965
LCID,NaN,5.287925e+07,5.160079,0.000000,0.091175
LULU,NaN,5.596278e+05,16.551056,0.000000,0.019428
NVDA,NaN,1.378364e+07,11.965345,0.008729,0.039455
SPY,NaN,3.376680e+07,5.916319,0.000000,0.010747


In [46]:
#annualized volatility
yq_data2['stdev'] = yq_data2['logret'] * 252**.5

In [47]:
yq_data2.reset_index(inplace=True)

In [48]:
#dict with stdev
vol_map = dict(yq_data2[['symbol','stdev']].values)

In [49]:
# do same as above but n days should be longer, 30 days vs 90 for example

d90 = datetime.today() - timedelta(days=90)
yq_data90x = stocks2.history(start=d90, interval='1d')

yq_data90x.reset_index(inplace=True)

yq_data90 = yq_data90x.drop(columns = ['open', 'low', 'close', 'high', 'dividends'])
yq_data90 = yq_data90.assign(logret=np.log(yq_data90.adjclose).groupby(yq_data90.symbol).diff()) 

yq_data2_90 =pd.DataFrame()

yq_data2_90 = yq_data90.groupby('symbol').agg(np.std, ddof=1) 

yq_data2_90['stdev'] = yq_data2_90['logret'] * 252**.5
yq_data2_90.reset_index(inplace=True)
vol_map90 = dict(yq_data2_90[['symbol','stdev']].values)

In [50]:
yq_data90

,symbol,date,volume,adjclose,logret
0,CHWY,2021-09-13,3005800,73.639999,NaN
1,CHWY,2021-09-14,1771700,72.220001,-0.019471
2,CHWY,2021-09-15,2446400,73.650002,0.019607
3,CHWY,2021-09-16,2221200,74.919998,0.017097
4,CHWY,2021-09-17,2548300,75.620003,0.009300
...,...,...,...,...,...
625,LCID,2021-12-03,76351400,47.270000,-0.023831
626,LCID,2021-12-06,103929800,44.860001,-0.052329
627,LCID,2021-12-07,57740800,43.880001,-0.022088
628,LCID,2021-12-08,50135900,44.720001,0.018962


In [51]:
yq_data3 =pd.DataFrame()

yq_data3 = yq_data.groupby('symbol')['volume'].agg(np.mean).to_dict()

In [52]:
# map new columns

df['h_volume'] = df.symbol.map(yq_data3)
df['volume_today'] = df.symbol.map(today_volume)
df['hvol_30'] = df.symbol.map(vol_map)
df['hvol_90'] = df.symbol.map(vol_map90)

In [53]:
df['vT/vH'] =  df['volume_today'] / df['h_volume'] 

In [54]:
#mean sigma with weights

df['mean_sigma'] =  .65*df['hvol_30'] + .2*df['hvol_90'] +.15*df['impliedVolatility'] 
df['ba_spread'] =  ((df['ask'] - df['bid'])/df['ask']).abs()


In [55]:
df = df[df['ba_spread'] > 0]

In [56]:
#create option prices using BS function
df['option_bs_mean_S'] = df.apply(lambda row: optionBS_price(row['optionType'],row['stock_price'], row['strike'],row['mean_sigma'],row['dte']), axis=1)

<ipython-input-56-090172e8b113>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['option_bs_mean_S'] = df.apply(lambda row: optionBS_price(row['optionType'],row['stock_price'], row['strike'],row['mean_sigma'],row['dte']), axis=1)


In [57]:
df

,symbol,expiration,optionType,Distance OTM,contractSymbol,strike,lastPrice,volume,bid,ask,...,dte,Abs_Dist/Price,h_volume,volume_today,hvol_30,hvol_90,vT/vH,mean_sigma,ba_spread,option_bs_mean_S
0,CHWY,2021-12-10,calls,26.30,CHWY211210C00030000,30.0,27.06,35.0,25.95,27.65,...,0,0.467140,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.713630,0.061483,26.300000
1,CHWY,2021-12-10,calls,21.30,CHWY211210C00035000,35.0,22.51,1.0,21.25,23.65,...,0,0.378330,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.703082,0.101480,21.300000
2,CHWY,2021-12-10,calls,16.30,CHWY211210C00040000,40.0,16.83,34.0,16.20,17.55,...,0,0.289520,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.267145,0.076923,16.300000
3,CHWY,2021-12-10,calls,11.30,CHWY211210C00045000,45.0,12.69,112.0,11.90,13.00,...,0,0.200710,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.200934,0.084615,11.300000
4,CHWY,2021-12-10,calls,6.30,CHWY211210C00050000,50.0,8.35,195.0,8.20,8.55,...,0,0.111901,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.126520,0.040936,6.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27947,LCID,2024-01-19,puts,-28.48,LCID240119P00065000,65.0,37.35,1.0,36.50,41.40,...,770,0.779847,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.296502,0.118357,48.515789
27948,LCID,2024-01-19,puts,-33.48,LCID240119P00070000,70.0,39.30,2.0,41.00,45.50,...,770,0.916758,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.296282,0.098901,52.999494
27949,LCID,2024-01-19,puts,-38.48,LCID240119P00075000,75.0,46.47,1.0,45.00,49.90,...,770,1.053669,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.294909,0.098196,57.495255
27950,LCID,2024-01-19,puts,-43.48,LCID240119P00080000,80.0,50.42,2.0,49.50,54.40,...,770,1.190581,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.295257,0.090074,62.051295


In [58]:
from MACD import rollavg
Price_roll = rollavg(stocks,yq_data90x,30,"Price",MACD_P=True)

In [59]:
Volume_roll = rollavg(stocks,yq_data90x,30,"Volume",MACD_V=True)

In [60]:
df['MACD_P'] = df['symbol'].map(Price_roll)
df['MACD_V'] = df['symbol'].map(Volume_roll)

<ipython-input-60-f12c4c3ec454>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_P'] = df['symbol'].map(Price_roll)
<ipython-input-60-f12c4c3ec454>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_V'] = df['symbol'].map(Volume_roll)


In [61]:
df


,symbol,expiration,optionType,Distance OTM,contractSymbol,strike,lastPrice,volume,bid,ask,...,h_volume,volume_today,hvol_30,hvol_90,vT/vH,mean_sigma,ba_spread,option_bs_mean_S,MACD_P,MACD_V
0,CHWY,2021-12-10,calls,26.30,CHWY211210C00030000,30.0,27.06,35.0,25.95,27.65,...,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.713630,0.061483,26.300000,0.956626,1.122203
1,CHWY,2021-12-10,calls,21.30,CHWY211210C00035000,35.0,22.51,1.0,21.25,23.65,...,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.703082,0.101480,21.300000,0.956626,1.122203
2,CHWY,2021-12-10,calls,16.30,CHWY211210C00040000,40.0,16.83,34.0,16.20,17.55,...,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.267145,0.076923,16.300000,0.956626,1.122203
3,CHWY,2021-12-10,calls,11.30,CHWY211210C00045000,45.0,12.69,112.0,11.90,13.00,...,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.200934,0.084615,11.300000,0.956626,1.122203
4,CHWY,2021-12-10,calls,6.30,CHWY211210C00050000,50.0,8.35,195.0,8.20,8.55,...,4.035070e+06,8097075,0.621758,0.529853,2.006675,1.126520,0.040936,6.300000,0.956626,1.122203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27947,LCID,2024-01-19,puts,-28.48,LCID240119P00065000,65.0,37.35,1.0,36.50,41.40,...,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.296502,0.118357,48.515789,0.941125,0.998101
27948,LCID,2024-01-19,puts,-33.48,LCID240119P00070000,70.0,39.30,2.0,41.00,45.50,...,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.296282,0.098901,52.999494,0.941125,0.998101
27949,LCID,2024-01-19,puts,-38.48,LCID240119P00075000,75.0,46.47,1.0,45.00,49.90,...,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.294909,0.098196,57.495255,0.941125,0.998101
27950,LCID,2024-01-19,puts,-43.48,LCID240119P00080000,80.0,50.42,2.0,49.50,54.40,...,9.819057e+07,119660973,1.447356,1.174261,1.218660,1.295257,0.090074,62.051295,0.941125,0.998101


In [62]:

slope = pd.pivot_table(df, index = ['symbol','expiration'], columns='strike', values = 'impliedVolatility', aggfunc='mean').reset_index(level=1)

In [63]:
slope.reset_index(inplace=True)

In [64]:
slope

strike,symbol,expiration,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,...,3300.0,3400.0,3500.0,3600.0,3700.0,3800.0,3900.0,4000.0,4100.0,4200.0
0,AAPL,2021-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,2021-12-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAPL,2021-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAPL,2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,TSLA,2022-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.73669,1.344951,1.323901,1.30416,1.286452,1.277579,2.507641,2.478344,1.225818,1.212915
170,TSLA,2023-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,TSLA,2023-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,TSLA,2023-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:

def slope_reg(i):
    #for i in range(len(slope.index)):
        slopex=slope.iloc[[i]].dropna(1)
        slopex1 = slopex.T
        x =slopex1.index[2:].to_numpy()
        x=x.astype(float)
        y = slopex1.iloc[2:,0:].to_numpy()
        y = y.ravel()
        y=y.astype(float)
        slopeB = stats.linregress(x, y)
        return slopeB.slope

In [66]:
slope['slope_reg'] = slope.apply(lambda row: slope_reg(row.name), axis=1)

In [67]:
slope

strike,symbol,expiration,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,...,3400.0,3500.0,3600.0,3700.0,3800.0,3900.0,4000.0,4100.0,4200.0,slope_reg
0,AAPL,2021-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.021167
1,AAPL,2021-12-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.012710
2,AAPL,2021-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002786
3,AAPL,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004388
4,AAPL,2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,TSLA,2022-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.344951,1.323901,1.30416,1.286452,1.277579,2.507641,2.478344,1.225818,1.212915,0.000158
170,TSLA,2023-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000126
171,TSLA,2023-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000175
172,TSLA,2023-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000158


In [68]:
# to check for specific ticker 
#slope.loc[slope.symbol == 'BB']

In [69]:

slope.groupby(['symbol'])['slope_reg'].mean().to_frame().reset_index().sort_values(['slope_reg'], ascending=False)

,symbol,slope_reg
1,AMC,0.005708
5,LCID,0.004375
4,GME,0.002458
9,TSLA,0.000002
6,LULU,-0.000745
2,AMD,-0.000795
7,NVDA,-0.001159
3,CHWY,-0.001343
8,SPY,-0.001890
0,AAPL,-0.002764


In [70]:
slope.groupby(['symbol'])['slope_reg'].apply(lambda x : x.head(3).reset_index(drop = True)).groupby(['symbol']).mean().sort_values(ascending=False).head(20).to_frame().reset_index()

,symbol,slope_reg
0,AMC,0.023230
1,LCID,0.018051
2,GME,0.007526
3,CHWY,0.002819
4,TSLA,0.000199
5,LULU,-0.000956
6,AMD,-0.001692
7,NVDA,-0.002768
8,SPY,-0.003860
9,AAPL,-0.012221
